In [ ]:
import pandas as pd
import spacy 

In [ ]:
df2 = pd.read_csv('WikiLarge_Test.csv') # will be test when run it
concreteness = pd.read_csv(r'C:\Users\corsi\Downloads\umich-siads-695-fall21-predicting-text-difficulty\Concreteness_ratings_Brysbaert_et_al_BRM.txt', delimiter = "\t")
basic = pd.read_csv(r'C:\Users\corsi\Downloads\umich-siads-695-fall21-predicting-text-difficulty\dale_chall.txt', delimiter = "\t", header=None)
aol =pd.read_csv(r'C:\Users\corsi\Downloads\umich-siads-695-fall21-predicting-text-difficulty\AoA_51715_words.csv', header=0,encoding = "ISO-8859-1" )

In [ ]:
import en_core_web_lg
nlp_model = en_core_web_lg.load()
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp_model.vocab)

In [ ]:
def lemmatize_text_no_punc(text): #lemma_word1.append(token.lemma_)
    text = " ".join([w.lower() for w in text.split()]) # may not need it's lowercasing beforing sending to tokenizer appears to make some difference like References 
    return " ".join([w.lemma_.lower() for w in tokenizer(text) if nlp_model.vocab[w.lemma_].is_punct == False])

def lemmatize_text(text): #lemma_word1.append(token.lemma_)
    text = " ".join([w.lower() for w in text.split()]) # may not need it's lowercasing beforing sending to tokenizer appears to make some difference like References 
    return " ".join([w.lemma_.lower() for w in tokenizer(text)])

In [ ]:
basic.rename(columns={0:'Word'},inplace=True)
basic['Word'] = basic['Word'].astype(str)
basic['Word_lem'] = basic['Word'].apply(lemmatize_text)

concreteness['Word'] = concreteness['Word'].astype(str)
concreteness['Word_lem'] = concreteness['Word'].apply(lemmatize_text)

aol['Word'] = aol['Word'].astype(str)
aol['Word_lem'] = aol['Word'].apply(lemmatize_text)

df['text_lemma'] = df['original_text'].apply(lemmatize_text)
df['text_lemma_no_punc'] = df['original_text'].apply(lemmatize_text_no_punc)

In [ ]:
basic_word_set = set(basic['Word_lem'].to_list())

def basic_rank(text): # could also potentially reduce to stem to check as well stampeders should map to stamp
  split_sent = text.split()
  split_len = len(split_sent)
  count = 0
  count = len(basic_word_set.intersection(set(split_sent)))
  # for i in split_sent:
  #   if len(basic_word_set.intersection(i)) == 1:
  #     count+=1
  try:
    return count/split_len
  except:
    return 0

def oov(text): # higher less words oov
  split_sent = text.split()
  split_len = len(split_sent)
  count = 0
  for i in split_sent:
    if nlp_model.vocab[i].is_oov == False:
      count+=1
  
  try:
    return count/split_len
  except:
    return 0

In [ ]:
df['basic_rank'] = df['text_lemma'].apply(basic_rank)
df['basic_rank_no_punc'] = df['text_lemma_no_punc'].apply(basic_rank) # could be more accurate because not diluted by punctuation

df['oov_rank'] = df['text_lemma'].apply(oov)
df['oov_rank_no_punc'] = df['text_lemma_no_punc'].apply(oov)

In [ ]:
from collections import defaultdict
d = defaultdict(float, {'a':3.23423})

In [ ]:
concreteness_dict = dict(zip(concreteness['Word_lem'],concreteness['Conc.M'] )) # should also try scaling by perc known can multiple right here when creating initial dict
concreteness_dict_perc = dict(zip(concreteness['Word_lem'],concreteness['Conc.M']*concreteness['Percent_known'] )) # should also try scaling by perc known can multiple right here when creating initial dict
def concreteness_rank(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, concreteness_dict)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan

  average_array = np.nanmean(text_array)


  return average_array

def concreteness_rank_nan(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, concreteness_dict)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan
  text_array[text_array == 0.0] = np.nan
  average_array = np.nanmean(text_array)


  return average_array

def concreteness_rank_perc(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, concreteness_dict_perc)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan

  average_array = np.nanmean(text_array)


  return average_array

def concreteness_rank_nan_perc(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, concreteness_dict_perc)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan
  text_array[text_array == 0.0] = np.nan
  average_array = np.nanmean(text_array)


  return average_array


In [ ]:
aol_dict = dict(zip(aol['Word_lem'],aol['AoA_Kup_lem'] )) 
def aol_rank(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, aol_dict)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan

  average_array = np.nanmean(text_array)


  return average_array


def aol_rank_nan(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, aol_dict)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan
  text_array[text_array == 0.0] = np.nan
  average_array = np.nanmean(text_array)


  return average_array


aol_dict_perc = dict(zip(aol['Word_lem'],aol['AoA_Kup_lem']*aol['Perc_known'] )) 
def aol_rank_perc(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, aol_dict_perc)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan

  average_array = np.nanmean(text_array)


  return average_array


def aol_rank_nan_perc(text):
  import numpy as np
  from collections import defaultdict
  temp_dict = defaultdict(float, aol_dict_perc)
  split_sent = text.split()

  text_array = np.array([temp_dict[i] for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan
  text_array[text_array == 0.0] = np.nan
  average_array = np.nanmean(text_array)


  return average_array

In [ ]:
def sent_length(text):
  return len(text)

def word_count(text):
  return len(text.split())

def word_length(text): #original text

  import numpy as np
  split_sent = [w for w in tokenizer(text) if nlp_model.vocab[w.lemma_].is_punct == False]
  

  text_array = np.array([len(i) for i in split_sent]) # if not present substitute 0 should think about making nan ht = defaultdict(lambda:np.Nan, ht), arr[arr == 0] = 'nan' # or use np.nan
  average_array = np.nanmean(text_array)


  return average_array

def stop_word(text): #original text
  split_sent = [w for w in tokenizer(text) if nlp_model.vocab[w.lemma_].is_stop == True]
  return len(split_sent)


def numbers(text): #original text
  split_sent = [w for w in tokenizer(text) if nlp_model.vocab[w.lemma_].is_digit == True]
  return len(split_sent)



def rare_punctuation(text):
  import string 
      
  # Storing the sets of punctuation in variable result 
  result = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ #'''# removed quotes and .
  
  split_sent = text.split()
  count = 0
  for i in split_sent:
    if i in result:
      count +=1

  return count



In [ ]:
df['word_len'] = df['original_text'].apply(word_length) #lemmatize will shorten only tokenize


df['stop_word'] = df['original_text'].apply(stop_word)
df['nums'] = df['original_text'].apply(numbers)

df['word_len_no_punc'] = df['text_lemma_no_punc'].apply(word_length)

df['sent_len'] = df['original_text'].apply(sent_length)

df['word_count_no_punct'] = df['text_lemma_no_punc'].apply(word_count)
df['word_count'] = df['text_lemma'].apply(word_count)


df['punct'] = df['original_text'].apply(rare_punctuation)
df['punct_lemm'] = df['text_lemma'].apply(rare_punctuation)

C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/1180071782.py:14: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)


In [ ]:
df['concreteness_rank_nan'] = df['text_lemma'].apply(concreteness_rank_nan)
df['concreteness_rank'] = df['text_lemma'].apply(concreteness_rank)

df['concreteness_rank_nan_no_punc'] = df['text_lemma_no_punc'].apply(concreteness_rank_nan)
df['concreteness_rank_no_punc'] = df['text_lemma_no_punc'].apply(concreteness_rank)

df['concreteness_rank_nan_perc'] = df['text_lemma'].apply(concreteness_rank_nan_perc)
df['concreteness_rank_perc'] = df['text_lemma'].apply(concreteness_rank_perc)

df['concreteness_rank_nan_no_punc_perc'] = df['text_lemma_no_punc'].apply(concreteness_rank_nan_perc)
df['concreteness_rank_no_punc_perc'] = df['text_lemma_no_punc'].apply(concreteness_rank_perc)



df['aol_rank_nan'] = df['text_lemma'].apply(aol_rank_nan)
df['aol_rank'] = df['text_lemma'].apply(aol_rank)

df['aol_rank_nan_no_punc'] = df['text_lemma_no_punc'].apply(aol_rank_nan)
df['aol_rank_no_punc'] = df['text_lemma_no_punc'].apply(aol_rank)


df['aol_rank_nan_perc'] = df['text_lemma'].apply(aol_rank_nan_perc)
df['aol_rank_perc'] = df['text_lemma'].apply(aol_rank_perc)

df['aol_rank_nan_no_punc_perc'] = df['text_lemma_no_punc'].apply(aol_rank_nan_perc)
df['aol_rank_no_punc_perc'] = df['text_lemma_no_punc'].apply(aol_rank_perc)

C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/1460970154.py:24: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)
C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/1460970154.py:11: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)
C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/1460970154.py:50: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)
C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/1460970154.py:37: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)
C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/3577557391.py:10: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)
C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/3577557391.py:53: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text_array)
C:\Users\corsi\AppData\Local\Temp/ipykernel_3696/3577557391.py:39: RuntimeWarning: Mean of empty slice
  average_array = np.nanmean(text

In [ ]:
df['text_lemma_no_punc'].isna().sum()

0

In [ ]:
>>> import nltk
>>> nltk.download('stopwords')
from tqdm import tqdm
import re
import gensim
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# YOUR CODE HERE
first_pass = []
tokenized_train_items = []
skip = set(stopwords.words('english'))

for bio in tqdm(df['text_lemma_no_punc']):
  try:
    temp = list(re.findall(r'\w+', bio))
    first_pass.append(temp)

    temp = [w.lower() for w in temp if w.lower() not in skip]
    tokenized_train_items.append(temp)
  except:
    tokenized_train_items.append('lemmanansnotext')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\corsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 119092/119092 [00:02<00:00, 52875.46it/s]


In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format("D:\Downloads\glove.6b\glove.6B.300d.txt", binary=False)

In [ ]:
def generate_dense_features(tokenized_texts, word_vectors): 
    #HINT: Create an empty list to hold your results 
        #HINT:Iterate through each item in tokenized_text
            #HINT:Create a list that contains current item(s) if found in word_vectors.vocab
            #HINT:if the length of this list is greater than zero:
                #HINT:We set this as a feature, this is done by using numpy’s mean function and append it to our results list 
            #HINT:Otherwise: create a vector of numpy zeros using word_vectors.vector_size as the parameter and append it to the results list
    #HINT:Return the results list as a numpy array (data type)
    import numpy as np
    # YOUR CODE HERE
    results = []
    for item in tokenized_texts:
        temp = [i for i in item if i in word_vectors.vocab]
        if len(temp) > 0 :
            results.append(np.mean(word_vectors[temp],axis=0)) #can also pass all words in at once, but did this method
            
        else:
            results.append(np.zeros(word_vectors.vector_size))
            
    return np.array(results)

In [ ]:
X_train_wp = generate_dense_features(tokenized_train_items, model)

In [ ]:
#shouldn't need this cell

# first_pass = []
# tokenized_train_items_dev = []
# skip = set(stopwords.words('english'))

# for bio in tqdm(dev_df['original_text']):
#     temp = list(re.findall(r'\w+', bio))
#     first_pass.append(temp)

#     temp = [w for w in temp if w.lower() not in skip]
#     tokenized_train_items_dev.append(temp)

# first_pass = []
# tokenized_train_items_test = []
# skip = set(stopwords.words('english'))

# for bio in tqdm(test['original_text']):
#     temp = list(re.findall(r'\w+', bio))
#     first_pass.append(temp)

#     temp = [w for w in temp if w.lower() not in skip]
#     tokenized_train_items_test.append(temp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41677/41677 [00:00<00:00, 58762.69it/s]


In [ ]:
# X_dev = generate_dense_features(tokenized_train_items_dev, model)
# X_test= generate_dense_features(tokenized_train_items_test, model)

In [ ]:
concreteness['scaled_m'] = concreteness['Conc.M']*concreteness['Percent_known']


In [ ]:
concreteness['quantiles'] = pd.qcut(concreteness['scaled_m'],labels=False, q=10)

In [ ]:
aol['scaled_m'] = aol['AoA_Kup_lem']*aol['Perc_known_lem']
aol['quantiles'] = pd.qcut(aol['scaled_m'],labels=False, q=10)

In [ ]:
top_quartiles_concreteness = concreteness[concreteness['quantiles'].isin([7,8,9])]['Word'].to_list()
middle_quartiles_concreteness = concreteness[concreteness['quantiles'].isin([4,5,6])]['Word'].to_list()
lower_quartiles_concreteness = concreteness[concreteness['quantiles'].isin([0,1,2,3])]['Word'].to_list()

In [ ]:
aol['scaled_m'] = aol['AoA_Kup_lem']*aol['Perc_known_lem']
aol['quantiles'] = pd.qcut(aol['scaled_m'],labels=False, q=10)

In [ ]:
top_quartiles_aol = aol[aol['quantiles'].isin([7,8,9])]['Word'].to_list()
middle_quartiles_aol = aol[aol['quantiles'].isin([4,5,6])]['Word'].to_list()
lower_quartiles_aol = aol[aol['quantiles'].isin([0,1,2,3])]['Word'].to_list()

In [ ]:
def quartile_similarity(tokenized_texts, quartile_list):

  import numpy as np
  from sklearn.metrics.pairwise import cosine_similarity
  top = [i for i in quartile_list[0] if i in model.vocab]
  mid = [i for i in quartile_list[1] if i in model.vocab]
  bottom = [i for i in quartile_list[2] if i in model.vocab]

  top_vec = np.mean(model[top],axis=0)
  mid_vec = np.mean(model[mid],axis=0)
  bottom_vec = np.mean(model[bottom],axis=0)


  # YOUR CODE HERE
  top_results = []
  mid_results = []
  bottom_results = []

  for item in tokenized_texts:
      temp = [i for i in item if i in model.vocab]
      if len(temp) > 0 :
          temp_res = np.mean(model[temp],axis=0)
          top_results.append(np.ravel(cosine_similarity(temp_res.reshape(-1,300,),top_vec.reshape(-1,300)))) 
          mid_results.append(np.ravel(cosine_similarity(temp_res.reshape(-1,300,),mid_vec.reshape(-1,300))))
          bottom_results.append(np.ravel(cosine_similarity(temp_res.reshape(-1,300,),bottom_vec.reshape(-1,300))))
          
      else:
          top_results.append(np.array([0.0]))
          mid_results.append(np.array([0.0]))
          bottom_results.append(np.array([0.0]))
          
  return top_results, mid_results, bottom_results


In [ ]:
top_aol,mid_aol,bot_aol = quartile_similarity(tokenized_train_items, [top_quartiles_aol, middle_quartiles_aol, lower_quartiles_aol])

In [ ]:
top_con,mid_con,bot_con = quartile_similarity(tokenized_train_items, [top_quartiles_concreteness, middle_quartiles_concreteness, lower_quartiles_concreteness])

In [ ]:
def quartile_similarity_single(tokenized_texts, quartile_list):

  import numpy as np
  from sklearn.metrics.pairwise import cosine_similarity
  top = [i for i in quartile_list if i in model.vocab]

  top_vec = np.mean(model[top],axis=0)


  # YOUR CODE HERE
  top_results = []
  mid_results = []
  bottom_results = []

  for item in tokenized_texts:
      temp = [i for i in item if i in model.vocab]
      if len(temp) > 0 :
          temp_res = np.mean(model[temp],axis=0)
          top_results.append(np.ravel(cosine_similarity(temp_res.reshape(-1,300,),top_vec.reshape(-1,300)))) 

          
      else:
          top_results.append(np.array([0.0]))

          
  return top_results

In [ ]:
basic_word = basic['Word'].to_list()
basic_sim = quartile_similarity_single(tokenized_train_items, basic_word)

In [ ]:
s =  ['top_aol','mid_aol','bot_aol','top_con','mid_con','bot_con','basic_sim']
for i in s:
  df[i] = eval(i)

In [ ]:
# import pandas as pd
# df = pd.read_csv(r'C:\Users\corsi\Downloads\umich-siads-695-fall21-predicting-text-difficulty\test_set_processed.csv')

In [ ]:
df['aol_rank_nan_perc']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
119087   NaN
119088   NaN
119089   NaN
119090   NaN
119091   NaN
Name: aol_rank_nan_perc, Length: 119092, dtype: float64

In [ ]:
objs = ['mid_con', 'bot_aol', 'mid_aol', 'top_aol', 'bot_con', 'top_con',
       'basic_sim']

def make_float(col):
  return (col)[0]
for i in objs:
  df[i] = df[i].apply(make_float)
for i in objs:
  df[i] = pd.to_numeric(df[i], errors='coerce')


In [ ]:
df.to_csv(r'C:\Users\corsi\Downloads\umich-siads-695-fall21-predicting-text-difficulty\test_set_processed.csv')

In [ ]:
df['bas']

,id,original_text,label,text_lemma,text_lemma_no_punc,basic_rank,basic_rank_no_punc,oov_rank,oov_rank_no_punc,word_len,...,aol_rank_nan_perc,aol_rank_perc,aol_rank_nan_no_punc_perc,aol_rank_no_punc_perc,top_aol,mid_aol,bot_aol,top_con,mid_con,bot_con
0,0,-2011,NaN,-2011,-2011,0.0,0.0,1.0,1.0,5.0,...,NaN,0.0,NaN,0.0,[-0.4457999],[-0.45882088],[-0.37928525],[-0.28325474],[-0.3743289],[-0.41504705]
1,1,-2011,NaN,-2011,-2011,0.0,0.0,1.0,1.0,5.0,...,NaN,0.0,NaN,0.0,[-0.4457999],[-0.45882088],[-0.37928525],[-0.28325474],[-0.3743289],[-0.41504705]
2,2,-2000,NaN,-2000,-2000,0.0,0.0,1.0,1.0,5.0,...,NaN,0.0,NaN,0.0,[-0.445449],[-0.45983702],[-0.3791357],[-0.2960482],[-0.36524928],[-0.40027258]
3,3,-1997,NaN,-1997,-1997,0.0,0.0,0.0,0.0,5.0,...,NaN,0.0,NaN,0.0,[-0.45167747],[-0.4718316],[-0.3959525],[-0.31601787],[-0.3771605],[-0.39861703]
4,4,1.636,NaN,1.636,1.636,0.0,0.0,0.0,0.0,5.0,...,NaN,0.0,NaN,0.0,[-0.2594516],[-0.2654487],[-0.19863749],[-0.09141784],[-0.21138589],[-0.29339764]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119087,119087,#NAME?,NaN,#name?,#name?,0.0,0.0,0.0,0.0,6.0,...,NaN,0.0,NaN,0.0,[-0.3035713],[-0.29423994],[-0.14190874],[-0.04912301],[-0.20984508],[-0.23277473]
119088,119088,#NAME?,NaN,#name?,#name?,0.0,0.0,0.0,0.0,6.0,...,NaN,0.0,NaN,0.0,[-0.3035713],[-0.29423994],[-0.14190874],[-0.04912301],[-0.20984508],[-0.23277473]
119089,119089,#NAME?,NaN,#name?,#name?,0.0,0.0,0.0,0.0,6.0,...,NaN,0.0,NaN,0.0,[-0.3035713],[-0.29423994],[-0.14190874],[-0.04912301],[-0.20984508],[-0.23277473]
119090,119090,#NAME?,NaN,#name?,#name?,0.0,0.0,0.0,0.0,6.0,...,NaN,0.0,NaN,0.0,[-0.3035713],[-0.29423994],[-0.14190874],[-0.04912301],[-0.20984508],[-0.23277473]


In [ ]:
df

,Unnamed: 0,original_text,label,text_lemma,text_lemma_no_punc,basic_rank,basic_rank_no_punc,oov_rank,oov_rank_no_punc,word_len,...,aol_rank_no_punc_perc,stop_word,nums,top_aol,mid_aol,bot_aol,top_con,mid_con,bot_con,basic_sim
0,0,There is manuscript evidence that Austen conti...,1,there be manuscript evidence that austen conti...,there be manuscript evidence that austen conti...,0.348837,0.394737,1.000000,1.000000,4.394737,...,3.110604,17,3,-0.394263,-0.380458,-0.248316,-0.201026,-0.274885,-0.276059,0.530072
1,1,"In a remarkable comparative analysis , Mandaea...",1,"in a remarkable comparative analysis , mandaea...",in a remarkable comparative analysis mandaean ...,0.304348,0.333333,0.869565,0.857143,6.190476,...,3.410769,7,0,0.026090,-0.024771,-0.064591,-0.127436,0.029195,0.115974,0.124346
2,2,"Before Persephone was released to Hermes , who...",1,"before persephone be release to hermes , who h...",before persephone be release to hermes who hav...,0.478261,0.523810,0.956522,0.952381,4.738095,...,3.921771,23,0,-0.227885,-0.169121,0.024210,0.025626,-0.037318,-0.163204,0.631895
3,3,Cogeneration plants are commonly found in dist...,1,cogeneration plant be commonly find in distric...,cogeneration plant be commonly find in distric...,0.435897,0.531250,1.000000,1.000000,6.281250,...,5.745808,6,0,-0.316036,-0.298752,-0.165115,-0.059336,-0.135955,-0.267575,0.585037
4,4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1,"geneva -lrb- , ; , ; , ; ; -rrb- be the second...",geneva -lrb- -rrb- be the second-most-populous...,0.250000,0.321429,0.777778,0.714286,5.642857,...,2.514700,12,0,-0.206032,-0.194760,-0.154996,-0.165122,-0.145344,-0.157995,0.191970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416763,416763,A Duke Nukem 3D version has been sold for Xbox...,0,a duke nukem 3d version have be sell for xbox ...,a duke nukem 3d version have be sell for xbox ...,0.470588,0.533333,1.000000,1.000000,4.066667,...,1.926667,5,2,-0.367020,-0.345096,-0.209537,-0.141144,-0.208176,-0.301036,0.453219
416764,416764,"However , it is becoming replaced as a method ...",0,"however , it be become replace a a method of e...",however it be become replace a a method of exe...,0.550000,0.611111,1.000000,1.000000,5.000000,...,5.725556,10,0,-0.191397,-0.221321,-0.186635,-0.172440,-0.069362,-0.090332,0.495821
416765,416765,There are hand gestures in both Hindu and Budd...,0,there be hand gesture in both hindu and buddhi...,there be hand gesture in both hindu and buddhi...,0.545455,0.600000,1.000000,1.000000,5.300000,...,5.422300,5,0,0.012151,0.023066,0.073097,0.053380,0.066035,0.089771,0.313336
416766,416766,"If it is necessary to use colors , try to choo...",0,"if it be necessary to use color , try to choos...",if it be necessary to use color try to choose ...,0.452381,0.487179,0.952381,0.948718,4.410256,...,4.231075,18,0,-0.128926,-0.091671,0.108809,0.165119,0.064642,-0.052814,0.689448
